In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import cv2
import numpy as np
import os
from scipy.interpolate import UnivariateSpline
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the source and target directories
source_dir = '/content/drive/MyDrive/cleaned_images'
target_dir = '/content/drive/MyDrive/ansys_importable_files'

# Make sure the target directory exists, if not, create it
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# List all image files in the source directory
image_files = [f for f in os.listdir(source_dir) if f.endswith('.png')]

# Loop through all image files
for image_file in image_files:
    # Load the image
    image_path = os.path.join(source_dir, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Binarize the image
    _, binary_image = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY_INV)

    # Find contours from the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour assuming it is the aerofoil
    aerofoil_contour = max(contours, key=lambda x: cv2.contourArea(x))

    # Ensure the contour is closed
    if not np.array_equal(aerofoil_contour[0], aerofoil_contour[-1]):
        aerofoil_contour = np.vstack((aerofoil_contour, aerofoil_contour[0][np.newaxis, :]))

    # Extract x and y coordinates
    x_coords, y_coords = aerofoil_contour[:, 0, 0], aerofoil_contour[:, 0, 1]

    # Correct the orientation by flipping the y-coordinates
    max_y = max(y_coords)
    corrected_y_coords = [max_y - y for y in y_coords]

    # Parametrize the curve by cumulative arc length
    arc_lengths = np.zeros(len(x_coords))
    for i in range(1, len(x_coords)):
        arc_lengths[i] = arc_lengths[i-1] + np.sqrt((x_coords[i] - x_coords[i-1])**2 + (corrected_y_coords[i] - corrected_y_coords[i-1])**2)
    arc_lengths /= arc_lengths[-1]

    # Adjust the smoothing factor
    smoothing_factor = 0.3 * len(x_coords)  # This value can be adjusted

    # Fit splines to the x and y coordinates as a function of the normalized arc length
    spline_x = UnivariateSpline(arc_lengths, x_coords, s=smoothing_factor)
    spline_y = UnivariateSpline(arc_lengths, corrected_y_coords, s=smoothing_factor)

    # Generate a smoothed curve
    smoothed_arc_length = np.linspace(0, 1, 500)
    smoothed_x = spline_x(smoothed_arc_length)
    smoothed_y = spline_y(smoothed_arc_length)

    # Ensure the curve is closed by setting the last point to the first
    smoothed_x[-1], smoothed_y[-1] = smoothed_x[0], smoothed_y[0]

    # Define the .dat file path
    dat_file_path = os.path.join(target_dir, os.path.splitext(image_file)[0] + '.dat')

    # Write the coordinates to the .dat file
    with open(dat_file_path, 'w') as file:
        file.write('! ANSYS data file created by custom script\n')
        file.write('/PREP7\n')
        file.write('! Define keypoints\n')
        for i, (x, y) in enumerate(zip(smoothed_x, smoothed_y)):
            file.write(f'K,{i+1}, {x}, {y}\n')

        file.write('! Define lines between keypoints\n')
        for i in range(len(smoothed_x)):
            file.write(f'L,{i+1},{(i+1) % len(smoothed_x) + 1}\n')

        file.write('FINISH\n')

print("Processing completed. Data files suitable for ANSYS import are saved in:", target_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing completed. Data files suitable for ANSYS import are saved in: /content/drive/MyDrive/ansys_importable_files


In [20]:
import matplotlib.pyplot as plt
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the target directory containing .dat files
target_dir = '/content/drive/MyDrive/ansys_importable_files'

# List all .dat files in the target directory
dat_files = [f for f in os.listdir(target_dir) if f.endswith('.dat')]

# Loop through all .dat files
for dat_file in dat_files:
    keypoints = []
    # Read the keypoints from the .dat file
    with open(os.path.join(target_dir, dat_file), 'r') as file:
        for line in file:
            if line.startswith('K,'):
                parts = line.split(',')
                # Extract the x and y coordinates
                x, y = float(parts[2].strip()), float(parts[3].strip())
                keypoints.append((x, y))

    # Convert the list of tuples into a list of X and Y coordinates for plotting
    x_coords, y_coords = zip(*keypoints)

    # Plot the keypoints
    plt.figure(figsize=(10, 5))
    plt.plot(x_coords, y_coords, '-o', label='Aerofoil Profile')
    plt.legend()
    plt.title('Aerofoil Profile Visualization')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.axis('equal')
    plt.grid(True)

    # Save the plot as an image file
    image_filename = os.path.splitext(dat_file)[0] + '.png'
    plt.savefig(os.path.join(target_dir, image_filename))
    plt.close()  # Close the figure to free memory

print("Visualization completed. Plots are saved in:", target_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Visualization completed. Plots are saved in: /content/drive/MyDrive/ansys_importable_files
